<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/ldaseq_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [81]:
import glob
from google.colab import drive
drive.mount('/content/drive')
##folder lists
ls = !ls "/content/drive/My Drive/words/sample"
ls = str(ls).strip("'[")
ls = ls.strip("]'")
entries = ls.split("\\t")
entries

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['month1', 'month2', 'month3']

In [91]:
## preparation for text preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [96]:
#loading corpus and preprocessing
corpus_list= []
for entry in entries:
    filepath = "/".join(["/content/drive/My Drive/words/sample", entry, "*.txt"])
    data_files = glob.glob(filepath)
    for data_file in data_files:
      text = open(data_file,'r').read().lower()
      filtered = [word for word in text.split() if not word in stop_words] ##revome stop words
      corpus = [ps.stem(word) for word in filtered] ##stem the word
      corpus_list += [corpus]
len(corpus_list)





1323

In [97]:
## create dictionary for distinct words


['pernod',
 'takeov',
 'talk',
 'lift',
 'domecq',
 'share',
 'uk',
 'drink',
 'food',
 'firm',
 'alli',
 'domecq',
 'risen',
 'specul',
 'could',
 'target',
 'takeov',
 "france'",
 'pernod',
 'ricard.',
 'report',
 'wall',
 'street',
 'journal',
 'financi',
 'time',
 'suggest',
 'french',
 'spirit',
 'firm',
 'consid',
 'bid,',
 'yet',
 'contact',
 'target.',
 'alli',
 'domecq',
 'share',
 'london',
 'rose',
 '4%',
 '1200',
 'gmt,',
 'pernod',
 'share',
 'pari',
 'slip',
 '1.2%.',
 'pernod',
 'said',
 'seek',
 'acquisit',
 'refus',
 'comment',
 'specifics.',
 "pernod'",
 'last',
 'major',
 'purchas',
 'third',
 'us',
 'giant',
 'seagram',
 '2000,',
 'move',
 'propel',
 'global',
 'top',
 'three',
 'drink',
 'firms.',
 'two-third',
 'seagram',
 'bought',
 'market',
 'leader',
 'diageo.',
 'term',
 'market',
 'value,',
 'pernod',
 '-',
 '7.5bn',
 'euro',
 '($9.7bn)',
 '-',
 '9%',
 'smaller',
 'alli',
 'domecq,',
 'capitalis',
 '£5.7bn',
 '($10.7bn;',
 '8.2bn',
 'euros).',
 'last',
 'yea

[['Books', 'are', 'on', 'the', 'table.'],
 ['Machine', 'Learning', 'is', 'cool!']]

312228

In [94]:
data_file = "/content/drive/My Drive/words/sample/month1/business002.txt"
text = open(data_file,'r').read().lower()
f =  [word for word in text.split() if not word in stop_words]
s = [ps.stem(word) for word in f ]
s



['dollar',
 'gain',
 'greenspan',
 'speech',
 'dollar',
 'hit',
 'highest',
 'level',
 'euro',
 'almost',
 'three',
 'month',
 'feder',
 'reserv',
 'head',
 'said',
 'us',
 'trade',
 'deficit',
 'set',
 'stabilise.',
 'alan',
 'greenspan',
 'highlight',
 'us',
 "government'",
 'willing',
 'curb',
 'spend',
 'rise',
 'household',
 'save',
 'factor',
 'may',
 'help',
 'reduc',
 'it.',
 'late',
 'trade',
 'new',
 'york,',
 'dollar',
 'reach',
 '$1.2871',
 'euro,',
 '$1.2974',
 'thursday.',
 'market',
 'concern',
 'deficit',
 'hit',
 'greenback',
 'recent',
 'months.',
 'friday,',
 'feder',
 'reserv',
 'chairman',
 'mr',
 "greenspan'",
 'speech',
 'london',
 'ahead',
 'meet',
 'g7',
 'financ',
 'minist',
 'sent',
 'dollar',
 'higher',
 'earlier',
 'tumbl',
 'back',
 'worse-than-expect',
 'us',
 'job',
 'data.',
 '"i',
 'think',
 "chairman'",
 'take',
 'much',
 'sanguin',
 'view',
 'current',
 'account',
 'deficit',
 "he'",
 'taken',
 'time,"',
 'said',
 'robert',
 'sinche,',
 'head',
 'cur